# AdaBoost
We fit an Adaboost classifer with the default decision tree to see if we could get a boost in our predictive ability. Adaboost works by fitting the data and then fitting copies of that data where the weights of the incorrectly classified instances are adjusted to narrow the models focus onto the more diffcult cases. We did achieve very good results but in the end did not beat our Logistic Regression model.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import pickle
import matplotlib.pyplot as plt


# Loading Time Series split assets

In [2]:
X_train = pd.read_pickle('../assets/X_train.pkl')
X_test = pd.read_pickle('../assets/X_test.pkl')
y_train = pd.read_pickle('../assets/y_train.pkl')
y_test = pd.read_pickle('../assets/y_test.pkl')

# Setting up a pipeline

In [3]:
pipe = Pipeline([
    ('ada',AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=100,min_samples_split=4))),
    
])


# Setting a range of hyperparameters


In [4]:
param_grid =  {
#  'ada__n_estimators':[],
 'ada__learning_rate':np.linspace(.001,.95,50)
}

# Running a gridsearch

In [5]:
gs = GridSearchCV(pipe, param_grid=param_grid,verbose=1,scoring='roc_auc', cv = TimeSeriesSplit())

In [6]:
gs.fit(X_train,y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-84bf55be5253>", line 1, in <module>
    gs.fit(X_train,y_train)
  File "/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py", line 639, in fit
    cv.split(X, y, groups)))
  File "/Applications/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 779, in __call__
    while self.dispatch_one_batch(iterator):
  File "/Applications/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 625, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Applications/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 588, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/Applications/anaconda3/lib/python3.6/site-packages/

KeyboardInterrupt: 

# Scoring

In [ ]:
gs.score(X_train,y_train)

In [ ]:
gs.score(X_test,y_test)

In [20]:
X_train_preds = gs.predict(X_train)

In [21]:
preds = pd.DataFrame({
    "preds":X_train_preds,
    "truth":y_train
})


In [22]:
preds.sum()

preds    258
truth    261
dtype: int64

In [23]:
gs.best_params_

{'ada__learning_rate': 0.7950612244897959, 'ada__n_estimators': 50}

In [24]:
y_train.shape

(6483,)

In [25]:
X_train.shape

(6483, 37)

In [26]:
X_train_preds.shape

(6483,)

In [27]:
with open('../assets/ada_boost_model.pkl','wb+') as f:
    pickle.dump(gs,f)